In [18]:
import crunchbase
from pprint import pprint
from dotenv import load_dotenv

load_dotenv()
crunchbase.BASE_CONFIG["cache"] = True

url = "https://www.crunchbase.com/organization/radai/people"
company = await crunchbase.scrape_company(url)

pprint(company)

2024-08-05 14:59:59.695 | INFO     | crunchbase:scrape_company:62 - scraping company: https://www.crunchbase.com/organization/radai/people


{'employees': [{'job_departments': ['marketing'],
                'job_levels': ['l_500_exec'],
                'linkedin': 'healthcarenerd',
                'name': 'Rikki Speck'},
               {'job_departments': ['management'],
                'job_levels': ['l_500_exec'],
                'linkedin': 'wboonn',
                'name': 'William Boonn, Md'},
               {'job_departments': ['management'],
                'job_levels': ['l_400_vp'],
                'linkedin': '1joshduncan',
                'name': 'Joshua Duncan'},
               {'job_departments': ['product'],
                'job_levels': ['l_400_vp'],
                'linkedin': 'sparasnis',
                'name': 'Shashank Parasnis'},
               {'job_departments': ['sales'],
                'job_levels': ['l_300_director'],
                'linkedin': 'brad-branum-8081ba14',
                'name': 'Brad Branum'},
               {'job_departments': None,
                'job_levels': ['l_300_director'],

In [19]:
pprint(f"""
# {company["organization"]["legal_name"]}, founded {company["organization"]["founded_on"]}
{company["organization"]["description"]}

Website: {company["organization"]["website"]}
LinkedIn: {company["organization"]["linkedin"]}
Twitter: {company["organization"]["twitter"]}
Facebook: {company["organization"]["facebook"]}

## Funding ({company["organization"]["funding_total_usd"] // 1000000}M total)

""")

for funding_round in sorted(company["organization"]["funding_rounds"], key=lambda round: round["announced_on"]):
    pprint(f"""
- {funding_round["raised_usd"] // 1000000}M on {funding_round["announced_on"]}
""")
    
print("News")
for article in company["organization"]["timeline"]:
    pprint(f"""
- {article["title"]} ({article["author"] or article["publisher"]}, {article["date"]})
""")

('\n'
 '# Rad AI, Inc., founded 2018-01-01\n'
 'Rad AI develops an artificial intelligence automated report generation '
 'platform to automate repetitive tasks for radiologists. Its platform '
 'integrates with existing workflow, automating repetitive tasks, and enabling '
 'radiologists to improve radiology workflow that maximizes efficiency and '
 'accuracy while reducing burnout for radiologists and improving patient '
 'care.\n'
 '\n'
 'Website: https://www.radai.com\n'
 'LinkedIn: https://www.linkedin.com/company/radai\n'
 'Twitter: https://twitter.com/radai\n'
 'Facebook: https://m.facebook.com/radaiFB\n'
 '\n'
 '## Funding (83M total)\n'
 '\n')
'\n- 8M on 2020-08-20\n'
'\n- 25M on 2021-11-10\n'
'\n- 50M on 2024-05-07\n'
News
('\n'
 '- Wake Radiology Adds Rad AI Technology to Revolutionize Radiology Reporting '
 '(Wake Radiology UNC Rex Healthcare, 2024-07-29)\n')
('\n'
 '- Bayer, Rad AI partner for AI radiology workflow offering (Mobi Health '
 'News, 2024-07-01)\n')
('\n'
 '- 

In [41]:
from datetime import date
from pydantic import BaseModel

from typing import List, Optional

class FundingRound(BaseModel):
    announced_on: date
    raised_usd: int
    investors: int
    lead_investors: List[str]

class Article(BaseModel):
    title: Optional[str]
    author: Optional[str]
    publisher: Optional[str]
    url: Optional[str]
    date: date
    type: str

class Organization(BaseModel):
    id: str
    name: str
    logo: str
    description: str
    founded_on: date
    linkedin: str
    facebook: str
    twitter: str
    email: str
    website: str
    ipo_status: str
    rank_org_company: int
    semrush_global_rank: int
    semrush_visits_latest_month: int
    semrush_id: str
    categories: List[str]
    legal_name: str
    operating_status: str
    funding_rounds: List[FundingRound]
    timeline: List[Article]

parsed = Organization(**company["organization"])

In [47]:
class Employee(BaseModel):
    job_departments: Optional[List[str]]
    job_levels: List[str]
    linkedin: str
    name: str

parsed_employees = [Employee(**emp) for emp in company["employees"]]
pprint(parsed_employees)

[Employee(job_departments=['marketing'], job_levels=['l_500_exec'], linkedin='healthcarenerd', name='Rikki Speck'),
 Employee(job_departments=['management'], job_levels=['l_500_exec'], linkedin='wboonn', name='William Boonn, Md'),
 Employee(job_departments=['management'], job_levels=['l_400_vp'], linkedin='1joshduncan', name='Joshua Duncan'),
 Employee(job_departments=['product'], job_levels=['l_400_vp'], linkedin='sparasnis', name='Shashank Parasnis'),
 Employee(job_departments=['sales'], job_levels=['l_300_director'], linkedin='brad-branum-8081ba14', name='Brad Branum'),
 Employee(job_departments=None, job_levels=['l_300_director'], linkedin='daltonjwright', name='Dalton Wright'),
 Employee(job_departments=['engineering'], job_levels=['l_300_director'], linkedin='jpaulett', name='John Paulett'),
 Employee(job_departments=None, job_levels=['l_300_director'], linkedin='matt-costello-3828741', name='Matt Costello'),
 Employee(job_departments=None, job_levels=['l_300_director'], linkedin

In [42]:
pprint(parsed)

Organization(id='radai', name='Rad AI', logo='ufzt8xwdhpqa47l3rinx', description='Rad AI develops an artificial intelligence automated report generation platform to automate repetitive tasks for radiologists. Its platform integrates with existing workflow, automating repetitive tasks, and enabling radiologists to improve radiology workflow that maximizes efficiency and accuracy while reducing burnout for radiologists and improving patient care.', founded_on=datetime.date(2018, 1, 1), linkedin='https://www.linkedin.com/company/radai', facebook='https://m.facebook.com/radaiFB', twitter='https://twitter.com/radai', email='info@radai.com', website='https://www.radai.com', ipo_status='private', rank_org_company=547, semrush_global_rank=1743709, semrush_visits_latest_month=13055, semrush_id='radai', categories=['Artificial Intelligence (AI)', 'Enterprise Software', 'Health Care', 'Medical'], legal_name='Rad AI, Inc.', operating_status='active', funding_rounds=[FundingRound(announced_on=datet

In [43]:
for article in sorted(parsed.timeline, key=lambda article: article.date):
    pprint(article)

Article(title='RAD AI Boosts Beauty Strategy with Maya Kosovalic, Appoints NYX Professional Makeup’s Digital Maverick to its Advisory Board', author=None, publisher='Business Wire', url='https://www.businesswire.com/news/home/20240326233387/en/RAD-AI-Boosts-Beauty-Strategy-with-Maya-Kosovalic-Appoints-NYX-Professional-Makeup%E2%80%99s-Digital-Maverick-to-its-Advisory-Board', date=datetime.date(2024, 3, 26), type='press_reference')
Article(title='RAD AI Announces Strategic Artificial Intelligence Buyout Plan to Propel Growth and Advance the Creator Economy', author=None, publisher='Morningstar', url='https://www.morningstar.com/news/business-wire/20240423009458/rad-ai-announces-strategic-artificial-intelligence-buyout-plan-to-propel-growth-and-advance-the-creator-economy', date=datetime.date(2024, 4, 23), type='press_reference')
Article(title=None, author=None, publisher=None, url=None, date=datetime.date(2024, 5, 7), type='funding_round')
Article(title='Rad AI Announces Partnership wit